# SHAP值

## SHAP值的基本原理
**SHAP(SHapley Additive exPlanations)**是一种基于博弈论的解释机器学习模型输出的方法。它的核心思想来源于博弈论中的Shapley值概念，用于公平地分配合作博弈中各个玩家的贡献。

在机器学习模型中：
- 每个特征被视为一个"玩家"
- 模型的预测结果被视为"游戏收益"
- SHAP值就是计算每个特征对预测结果的"边际贡献"

## SHAP值的主要特点：
   - **一致性**：如果模型改变使得某个特征更重要，其SHAP值会相应增加
   - **准确性**：所有特征的SHAP值之和等于模型预测与平均预测的差值
   - **可加性**：多个特征的SHAP值可以相加来理解它们的组合效应
   - **可解释性**：SHAP值方法特别适用于对深度神经网络等黑箱模型的解释

## SHAP值的优缺点

- **优点**

| 优点                   | 说明                           |
| -------------------- | ---------------------------- |
| **理论保证**             | 基于博弈论，具有公平性、一致性等数学性质         |
| **通用性**              | 可用于任意机器学习模型（如 tree、NN、SVM）   |
| **局部解释**             | 可以解释每一个单独预测值                 |
| **全局解释**             | 聚合 SHAP 值可提供特征重要性排名          |
| **一致性（Consistency）** | 如果模型改进后某特征的影响变大，其 SHAP 值也会一致变大 |

- **缺点**

| 缺点           | 说明                           |
| ------------ | ---------------------------- |
| **计算复杂**     | 计算成本指数级，消耗资源较大        |
| **特征依赖性问题**  | 默认特征相互独立，可能不适用于强相关特征         |
| **可解释性误用风险** | 对模型机制不了解可能误解 SHAP 的意义        |


## SHAP值的适用场景

| 场景       | 示例                        |
| -------- | ------------------------- |
| **模型调试** | 了解模型是否学习了错误特征             |
| **特征选择** | 根据全局 SHAP 值筛选重要特征         |
| **监管解释** | 银行/医疗等对可解释性有要求的行业         |
| **客户报告** | 向非技术人员解释模型为什么做出某个预测       |
| **异常检测** | 对 SHAP 值分布分析，发现异常样本或过拟合特征 |


# SHAP值手算详解

## 数据准备

### 信用卡审批预测模型

假设银行使用机器学习模型预测信用卡审批结果，模型考虑以下3个特征：

- **收入（Income）**：单位万元
- **负债率（DebtRatio）**：百分比
- **信用历史（CreditAge）**：年数
- 已知某申请人数据：Income=8, DebtRatio=30%, CreditAge=5
- 模型预测该申请人通过概率为65%。现需计算各特征的SHAP值贡献。

### 定义基准值和特征组合

**基准值（空集）**：模型对无任何特征的基准预测为40%（即$E[f(X)]=0.4$）

**所有特征组合**：
- {Income}
- {DebtRatio}
- {CreditAge}
- {Income, DebtRatio}
- {Income, CreditAge}
- {DebtRatio, CreditAge}
- {Income, DebtRatio, CreditAge}

**模型对各组合的预测值**

| 特征组合               | 预测概率 |
|------------------------|----------|
| ∅（空集）              | 40%      |
| {Income=8}               | 55%      |
| {DebtRatio=30}            | 35%      |
| {CreditAge=5}            | 45%      |
| {Income=8, DebtRatio=30}    | 60%      |
| {Income=8, CreditAge=5}    | 70%      |
| {DebtRatio=30, CreditAge=5} | 50%      |
| {Income=8, DebtRatio=30, CreditAge=5}               | 65%      |


## SHAP值手算方法——公式法（加权平均公式法）

### SHAP值的数学公式
SHAP值的计算公式如下：
$$\phi_i = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|!(|N|-|S|-1)!}{|N|!} \cdot \left[f(S \cup \{i\}) - f(S)\right]$$
其中：
- $\phi_i$：特征$i$的SHAP值
- $f$：机器学习模型
- $S$：除去特征$i$的特征子集
- $N$：所有特征集合
- $|N|$：总特征数量
- $f(S \cup \{i\})$：模型在特征子集$S$与特征$i$的预测值
- $f(S)$：模型在特征子集$S$的预测值

公式可以分解为三部分：
- **排列组合权重**：$\frac{|S|!(|N|-|S|-1)!}{|N|!}$ 表示不同特征顺序的重要性权重
- **边际贡献**：$\left[f(S \cup \{i\}) - f(S)\right]$表示加入特征$i$带来的预测变化
- **加权求和**：对所有可能的特征子集进行加权求和

### 公式权重求解

给定一个特征$i$，其 SHAP 值为：
$$\phi_i = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|!(|N|-|S|-1)!}{|N|!} \cdot \left[f(S \cup \{i\}) - f(S)\right]$$
- $N$：所有特征集合，这里是 {Income, DebtRatio, CreditAge} 
- 总特征数 $|N| = 3$，则 $3! = 6$
 
权重由子集大小决定：
 
- |S|=0：权重 = $\frac{0!2!}{6} = \frac{2}{6}$ 
- |S|=1：权重 = $\frac{1!1!}{6} = \frac{1}{6}$ 
- |S|=2：权重 = $\frac{2!0!}{6} = \frac{2}{6}$ 

### 分别计算 SHAP 值

#### Income 的 SHAP 值

子集$S \subseteq \{DebtRatio, CreditAge\}$

| 子集 S                 | 权重 | 边际贡献 f(S∪{Income})−f(S) |
|-------------------------|------|-------------------------------|
| ∅                       | 2/6  | 55% - 40% = +15%              |
| {DebtRatio}             | 1/6  | 60% - 35% = +25%              |
| {CreditAge}             | 1/6  | 70% - 45% = +25%              |
| {DebtRatio, CreditAge}  | 2/6  | 65% - 50% = +15%              |

$$\phi_{Income} = \frac{2}{6} \cdot 15 + \frac{1}{6} \cdot 25 + \frac{1}{6} \cdot 25 + \frac{2}{6} \cdot 15 = \frac{30 + 25 + 25 + 30}{6} = \frac{110}{6} ≈ 18.3\%$$

#### DebtRatio 的 SHAP 值

子集$S \subseteq \{Income, CreditAge\}$

| 子集 S              | 权重  | 边际贡献 f(S∪{DebtRatio})−f(S)         |
|---------------------|-------|---------------------------------------------|
| ∅                   | 2/6   | 35% - 40% = -5%                             |
| {Income}            | 1/6   | 60% - 55% = +5%                             |
| {CreditAge}         | 1/6   | 50% - 45% = +5%                             |
| {Income, CreditAge} | 2/6   | 65% - 70% = -5%                             |


$$\phi_{DebtRatio} = \frac{2}{6} \cdot (-5) + \frac{1}{6} \cdot 5 + \frac{1}{6} \cdot 5 + \frac{2}{6} \cdot (-5) = \frac{-10 + 5 + 5 -10}{6} = \frac{-10}{6} ≈ -1.7\%$$

#### CreditAge 的 SHAP 值

子集$S \subseteq \{Income, DebtRatio\}$

| 子集 S              | 权重  |  边际贡献 f(S∪{CreditAge})−f(S)$               |
|---------------------|-------|----------------------------------------------|
| ∅                   | 2/6   | 45% - 40% = +5%                              |
| {Income}            | 1/6   | 70% - 55% = +15%                             |
| {DebtRatio}         | 1/6   | 50% - 35% = +15%                             |
| {Income, DebtRatio} | 2/6   | 65% - 60% = +5%                              |

$$\phi_{CreditAge} = \frac{2}{6} \cdot 5 + \frac{1}{6} \cdot 15 + \frac{1}{6} \cdot 15 + \frac{2}{6} \cdot 5 = \frac{10 + 15 + 15 + 10}{6} = \frac{50}{6} ≈ 8.3\%$$

### 最终 SHAP 值结果

| 特征        | SHAP 值 |
| --------- | ------ |
| Income    | +18.3% |
| DebtRatio | -1.7%  |
| CreditAge | +8.3%  |

**加总验证**：
- $40\%（基准： ∅（空集）） + 18.3\% - 1.7\% + 8.3\% = 64.9\% ≈ 65\%$

✅ 与模型输出一致，验证计算正确！

## SHAP值手算方法——排列法（边际贡献法）

- 枚举所有特征加入顺序（共$n!$个排列）。
- 每一条顺序中，记录某个特征加入时对模型输出带来的边际贡献。
- 对该特征在所有顺序中的贡献取平均，就是该特征的 SHAP 值。

### 计算所有排列的边际贡献

对3个特征共有6种排列顺序，以下逐条计算：
1. 顺序 Income → DebtRatio → CreditAge
- **Income贡献**：
$ϕIncome=f(Income=8)−f(∅)=55\%−40\%=+15\%$
- **DebtRatio贡献**：
$ϕDebtRatio=f(Income=8, DebtRatio=30)−f(Income=8)=60\%−55\%=+5\%$
- **CreditAge贡献**：
$ϕCreditAge=f(全特征)−f(Income=8, DebtRatio=30)=65\%−60\%=+5\%$

2. 顺序 Income → CreditAge → DebtRatio
- **Income贡献**：
$ϕIncome=f(Income=8)−f(∅)=55\%−40\%=+15\%$
- **CreditAge贡献**：
$ϕCreditAge=f(Income=8, CreditAge=5)−f(Income=8)=70\%−55\%=+15\%$
- **DebtRatio贡献**：
$ϕDebtRatio=f(全特征)−f(Income=8, CreditAge=5)=65\%−70\%=−5\%$
3. 顺序 DebtRatio → Income → CreditAge
- **DebtRatio贡献**：
$ϕDebtRatio=f(DebtRatio=30)−f(∅)=35\%−40\%=−5\%$
- **Income贡献**：
$ϕIncome=f(Income=8, DebtRatio=30)−f(DebtRatio=30)=60\%−35\%=+25\%$
- **CreditAge贡献**：
$ϕCreditAge=f(全特征)−f(Income=8, DebtRatio=30)=65\%−60\%=+5\%$

4. 顺序 DebtRatio → CreditAge → Income
- **DebtRatio贡献**：
$ϕDebtRatio=f(DebtRatio=30)−f(∅)=35\%−40\%=−5\%$
- **CreditAge贡献**：
$ϕCreditAge=f(DebtRatio=30, CreditAge=5)−f(DebtRatio=30)=50\%−35\%=+15\%$
- **Income贡献**：
$ϕIncome=f(全特征)−f(DebtRatio=30, CreditAge=5)=65\%−50\%=+15\%$

5. 顺序 CreditAge → Income → DebtRatio
- **CreditAge贡献**：
$ϕCreditAge=f(CreditAge=5)−f(∅)=45\%−40\%=+5\%$
- **Income贡献**：
$ϕIncome=f(Income=8, CreditAge=5)−f(CreditAge=5)=70\%−45\%=+25\%$
- **DebtRatio贡献**：
$ϕDebtRatio=f(全特征)−f(Income=8, CreditAge=5)=65\%−70\%=−5\%$

6. 顺序 CreditAge → DebtRatio → Income
- **CreditAge贡献**：
$ϕCreditAge=f(CreditAge=5)−f(∅)=45\%−40\%=+5\%$
- **DebtRatio贡献**：
$ϕ DebtRatio=f(DebtRatio=30, CreditAge=5)−f(CreditAge=5)=50\%−45\%=+5\%$
- **Income贡献**：
$ϕIncome=f(全特征)−f(DebtRatio=30, CreditAge=5)=65\%−50\%=+15\%$


### 汇总贡献值并计算Shapley值

| 特征       | 所有排列中的贡献值             | 总和   | SHAP值（均值）         |
|------------|-------------------------------|--------|--------------------------|
| Income     | +15, +15, +25, +15, +25, +15  | 110%   | 110 / 6 ≈ 18.3%          |
| DebtRatio  | +5, -5, -5, -5, -5, +5        | -10%   | -10 / 6 ≈ -1.7%          |
| CreditAge  | +5, +15, +5, +15, +5, +5      | 50%    | 50 / 6 ≈ 8.3%            |

### 预测结果解释

基准值 + SHAP值总和应等于预测值：
`40%+18.3%−1.7%+8.3%=64.9%≈65%`
（与模型原始预测一致，验证计算正确）

- **Income**：正向贡献最大（+18.3%）
- **CreditAge**：中等正向贡献（+8.3%）
- **DebtRatio**：轻微负向影响（-1.7%）